![Logo](../assets/logo.png)

Made by **Domonkos Nagy**

[<img src="https://colab.research.google.com/assets/colab-badge.svg">](https://colab.research.google.com/github/Fortuz/rl_education/blob/main/1.%20K-armed%20Bandit/k_armed_bandit_solution.ipynb)

# $k$-armed Bandit (solution)

Consider the following learning problem. You are faced repeatedly with a choice among
$k$ different options, or actions. After each choice, you receive a numerical reward
from a stationary probability distribution that depends on the action you selected. Your
objective is to maximize the expected total reward over some time period, for example,
over 1000 action selections, or time steps.

This is called the $k$-armed bandit problem. You can visualize this problem as having to
choose between $k$ slot machines (also known as one-armed bandits) at each time step,
each of which has a different probability distribution for rewards - that is where the name comes from.

<img src="assets/k_armed_bandit.png" width="500"/>

The $k$-armed bandit problem illustrates an important problem in reinforcement
learning: **exploration vs. exploitation**. At each time step $t$, the agent has to make a decision:
take the action with the highest expected reward according to its current knowledge of the environment, 
or choose a different action to get a better estimation of the value of that action. The former is called an
*exploitation* step, because it exploits the current knowledge of the agent in order to obtain a high reward.
The latter is called an *exploration* step, since it involves trying out an action in order to have a better
estimation of its value, thereby exploring the environment.

This notebook introduces a few common strategies to tackle this problem and puts them to the test by simulating
multiple test runs, and comparing the results.

- This notebook is based on Chapter 2 of the book *Reinforcement Learning: An Introduction (2nd ed.)* by R. Sutton & A. Barto, available at http://incompleteideas.net/book/the-book-2nd.html

In [1]:
import numpy as np
from abc import ABC, abstractmethod
import matplotlib.pyplot as plt
from tqdm.notebook import trange
import time

## Strategy setup

The `Strategy` base class is used to implement startegies for action selection. An action is selected by the `act` method, and then the `update` method is used
to update the inner state after receiving a reward for the selected action. After an episode (a "run" consisting of $n$ steps, 1000 for example) is over, the `reset` method is called to reset the inner state of the class. The `name` propery is used get a name for the strategy in a visual representation.

In [2]:
# DO NOT MODIFY THIS CELL

class Strategy(ABC):

    def __init__(self, k):
        self.k = k  # Number of actions

    @property
    @abstractmethod
    def name():
        pass

    @abstractmethod
    def act(self):
        pass

    @abstractmethod
    def update(self, action, reward):
        pass

    @abstractmethod
    def reset(self):
        pass

Numpy's `np.argmax` will choose the smallest index in case there are multiple
maximal values. This function breaks these ties randomly instead, which is
desirable in many cases.

In [3]:
# DO NOT MODIFY THIS CELL

# Argmax function that breaks ties randomly
def argmax(arr):
    arr_max = np.max(arr)
    return np.random.choice(np.where(arr == arr_max)[0])

## Environment Setup

We test our strategies by trying them out in multiple runs, and then averaging out the received reward at each time step. After that, we plot the results to
compare the strategies.

In [ ]:
# DO NOT MODIFY THIS CELL

# Action with some probability distribution
class Action:
    def __init__(self, distribution_prob=[1, 0, 0]):
        self.mean = np.random.randn()
        # By default, the distribution is Normal
        self.distribution = np.random.choice(["normal", "constant", "uniform"], p=distribution_prob)

    def get_reward(self):
        if self.distribution == "normal":
            return np.random.randn() + self.mean
        elif self.distribution == "uniform":
            return np.random.rand() + self.mean
        else:
            return self.mean


# Simulating the k-armed bandit problem
def simulate(strategies, K, runs=2000, n_steps=1000):
    rewards = np.zeros((len(strategies), runs, n_steps))
    best_action_choices = np.zeros(rewards.shape)

    for i, strategy in enumerate(strategies):
        print(f"Evaluating strategy {i + 1}/{len(strategies)}...")

        for r in trange(runs):
            # Setting up actions with random probabilty distributions
            actions = [Action() for _ in range(K)]
            best_action = np.argmax([action.mean for action in actions])

            strategy.reset()

            for t in range(n_steps):
                action = strategy.act()

                reward = actions[action].get_reward()
                rewards[i, r, t] = reward
                strategy.update(action, reward)

                if action == best_action:
                    best_action_choices[i, r, t] = 1
                    
            time.sleep(.00001)  # For tqdm
            
    # Average of rewards received over all runs
    mean_rewards = rewards.mean(axis=1)
    # Average of number of best action choices over all runs
    mean_best_action_choices = best_action_choices.mean(axis=1)

    return mean_rewards, mean_best_action_choices

To examine the results a plot function is defined.

In [ ]:
# DO NOT MODIFY THIS CELL

def plotResults(strategies, rewards, best_action_choices):
  fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(14, 5))

  for strategy, reward in zip(strategies, rewards):
      ax1.plot(reward, label=f"{strategy.name}", zorder=2)
  ax1.set_xlabel('Steps')
  ax1.set_ylabel('Average reward')
  ax1.grid(alpha=0.8, linestyle=':', zorder=0)
  ax1.set_title('Average reward of strategies')
  ax1.legend()

  for strategy, choices in zip(strategies, best_action_choices):
      ax2.plot(choices, label=f"{strategy.name}")
  ax2.set_xlabel('Steps')
  ax2.set_ylabel('% Optimal action')
  ax2.grid(alpha=0.8, linestyle=':', zorder=0)
  ax2.set_title('% Optimal action choices of strategies')
  ax2.legend()

  plt.show()

## $\varepsilon$-greedy Action Selection

With this method, the agent will select a random action with an $\varepsilon$ probability ($0 \le \varepsilon \le 1$), and act greedily (select the best action according to its knowledge) otherwise. The action values are calculated using the *sample-averages* method: the value of an action is the average of all the rewards received after taking that action.

***

### **Your Task**

Implement the $\varepsilon$-greedy action selection method! You should complete all methods of the class below, and
you can add new class methods and constructor arguments as needed. Pseudocode for this algorithm is shown in the box below.

<img src="assets/epsilon_greedy.png" width="700"/>

*Pseudocode from page 32 of the Sutton & Barto book*

#### **Hints:**

- Use the argmax function defined above!
- There is a simple way to incremetally calculate the average reward for a given action: $Q_{n+1} = Q_n + \frac{1}{n}\left[R_n-Q_n\right]$ (See page 31 of the Sutton & Barto book for details)
- To test your implementation, instantiate this class in the `strategies` list at the bottom of the notebook and run the simulation.
- Try experimenting with different values of $\varepsilon$!

***

In [ ]:
class EpsilonGreedy(Strategy):

    def __init__(self, k, epsilon=0, initial=0):
        super().__init__(k)
        self.epsilon = epsilon  # Probability of exploration
        self.initial = initial  # Initial action values
        self.q_estimations = np.zeros(self.k) + self.initial  # Action value estimations
        self.selections = np.zeros(self.k)  # Number of selection of each action
        self.indices = np.arange(self.k)  # List of possible actions

    @property
    def name(self):
        name_str = ""
        if (self.epsilon == 0):
            name_str = "Greedy"
        else:
            name_str = f"$\\varepsilon$-greedy, $\\varepsilon = {self.epsilon}$"
        if (self.initial != 0):
            name_str += f", init: {self.initial}"
        return name_str

    def act(self):
        ############## CODE HERE ###################
        # Explore with probability epsilon
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.indices)
        # Exploit otherwise
        else:
            return argmax(self.q_estimations)
        ############################################

    # Sample-averages method for calculating action values
    def update(self, action, reward):
        ############## CODE HERE ###################
        self.selections[action] += 1
        # An incremental way of calculating the average
        # See pg. 31 of the Sutton-Barto book for more details
        self.q_estimations[action] += (reward - self.q_estimations[action]) / self.selections[action]
        ############################################

    # Reset inner state
    def reset(self):
        self.q_estimations = np.zeros(self.k) + self.initial
        self.selections = np.zeros(self.k)

Test of $\varepsilon$-greedy Action Selection.

In [ ]:
K = 3  # Number of actions

# List of strategies to test
strategies = [EpsilonGreedy(K)]

# Evaluate strategies
rewards, best_action_choices = simulate(strategies, K=K, runs=200, n_steps=200)

plotResults(strategies, rewards, best_action_choices)

## Upper-Confidence-Bound (UCB) Action Selection

The UCB action selection method offers a way to select an action by taking both the estimated value, as well as the accuracy of those estimates into account.
It uses the following formula:

$$ A_t := \underset{a}{\arg\max} \left[ Q_t(a) + c \sqrt{\frac{\ln(t)}{N_t(a)}} \right] $$

Where $Q_t(a)$ denotes the value of action $a$ (calculated using the *sample-averages* method), $N_t(a)$ denotes the number of times that action $a$ has
been selected prior to time $t$, and the number $c > 0$ controls
the degree of exploration. If $N_t(a) = 0$, then $a$ is considered to be a maximizing action.

***

### **Your Task**

Implement the UCB action selection method! You should complete all methods of the class below, and
you can add new class methods and constructor arguments as needed.

#### **Hints:**

- You can utilize the same incremental average calulation as with the $\varepsilon$-greedy method.
- Compare UCB to $\varepsilon$-greedy by adding an instance of this class to the `strategies` list below.
- Try experimenting with different values of $c$!

***

In [ ]:
class UCB(Strategy):

    def __init__(self, k, c=1, initial=0):
        super().__init__(k)
        self.c = c  # Degree of exploration
        self.initial = initial  # Initial action values
        self.q_estimations = np.zeros(self.k) + self.initial  # Action value estimations
        self.selections = np.zeros(self.k)  # Number of selections of each action
        self.t = 0  # Time step

    @property
    def name(self):
        return f"UCB, $c = {self.c}$"

    def act(self):
        ############## CODE HERE ###################
        # Calculate UCB estimations, take argmax
        UCB_estimations = self.q_estimations + self.c * np.sqrt(np.log(self.t + 1) / (self.selections + 1e-5))
        return argmax(UCB_estimations)
        ############################################

    # Sample-averages method for calculating action values
    def update(self, action, reward):
        ############## CODE HERE ###################
        self.t += 1
        self.selections[action] += 1
        # An incremental way of calculating the average
        # See pg. 31 of the Sutton-Barto book for more details
        self.q_estimations[action] += (reward - self.q_estimations[action]) / self.selections[action]
        ############################################

    # Reset inner state
    def reset(self):
        self.t = 0
        self.q_estimations = np.zeros(self.k) + self.initial
        self.selections = np.zeros(self.k)

Test of UCB.

In [ ]:
K = 3  # Number of actions

# List of strategies to test
strategies = [UCB(K)]

# Evaluate strategies
rewards, best_action_choices = simulate(strategies, K=K, runs=200, n_steps=200)

plotResults(strategies, rewards, best_action_choices)

## Gradient Bandit Algorithms

Instead of estimating action values, this method learns a numerical *preference*, denoted $H_t(a)$ for each action. The larger the preference, the more often that action is taken, but the preference has no interpretation in terms of reward. Action probabilites are determined using the *soft-max* function:

$$ \pi_t(a) := \Pr\{A_t = a\} := \frac{e^{H_t(a)}}{\sum_{b=1}^k e^{H_t(b)}} $$

Here we have also introduced a useful new notation, $\pi_t(a)$, for the probability of
taking action $a$ at time $t$. Note that this function defines a probability distribution over the set of all actions. On each step, after selecting action $A_t$ and receiving the reward $R_t$, the
action preferences are updated by:

$$ H_{t+1}(a) := H_t(a) + \alpha(R_t - \bar{R}_t)(\mathbb{1}_{a=A_t} - \pi_t(a)) $$

Where $\alpha > 0$ is a step-size parameter, and $\bar{R}_t \in \mathbb{R}$ is the average of all the rewards up
through and including time $t$. 
The $\bar{R}_t$ term serves as a
baseline with which the reward is compared.

***

### **Your Task**

Implement the Gradient action selection method! You should complete all methods of the class below, and
you can add new class methods and constructor arguments as needed.

#### **Hints:**

- You can incrementally calculate the baseline value similarly to the action-value averages for the methods above.
- The action preferences can be updated with a single line using an array which contains 1 at index $A_t$ and 0 elsewhere.
- Instantiate this class in the list below as well and see how the three methods compare!
- Try experimenting with different values of $\alpha$!

***

In [ ]:
class Gradient(Strategy):

    def __init__(self, k, alpha=0.1):
        super().__init__(k)
        self.alpha = alpha  # Step-size parameter
        self.preferences = np.zeros(self.k)  # Action preferences
        self.indices = np.arange(self.k)  # List of possible actions
        self.baseline = 0  # Average reward
        self.t = 0  # Time step

    @property
    def name(self):
        return f"Gradient, $\\alpha = {self.alpha}$"

    def act(self):
        ############## CODE HERE ###################
        # Select action from the softmax distribution
        return np.random.choice(self.indices, p=self.softmax(self.preferences))
        ############################################

    def update(self, action, reward):
        ############## CODE HERE ###################
        self.t += 1
        # Similar incremental calculation as the strategies above
        self.baseline += (reward - self.baseline) / self.t
        one_hot = np.zeros(self.k)
        one_hot[action] = 1
        self.preferences += self.alpha * (reward - self.baseline) * (one_hot - self.softmax(self.preferences))
        ############################################

    # Reset inner state
    def reset(self):
        self.t = 0
        self.baseline = 0
        self.preferences = np.zeros(self.k)

    def softmax(self, x):
        return np.exp(x) / sum(np.exp(x))

Test of Gradient Bandit Algorithm.

In [ ]:
K = 3  # Number of actions

# List of strategies to test
strategies = [EpsilonGreedy(K)]

# Evaluate strategies
rewards, best_action_choices = simulate(strategies, K=K, runs=200, n_steps=200)

plotResults(strategies, rewards, best_action_choices)

## Comprehensive test

In this final section let's run a longer comprehensive test with more actions.

In [ ]:
K = 5  # Number of actions

# List of strategies to test
strategies = [
        EpsilonGreedy(K),
        EpsilonGreedy(K, epsilon=0.1),
        UCB(K, c=2),
        Gradient(K)
    ]

# Evaluate strategies
rewards, best_action_choices = simulate(strategies, K=K, runs=2000, n_steps=500)

plotResults(strategies, rewards, best_action_choices)